In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm 
import os

# ==== Cihaz Ayarı ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Sabitler ====
BATCH_SIZE = 32
EPOCHS = 20
PATIENCE = 4
MODEL_PATH = "gul_resnet34.pth"

# ==== Veri Dönüştürücü ====
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder("/home/comp5/ARTEK/SYZ_25/ILK_GOREV/VERI/altair/train", transform=transform)
val_dataset   = datasets.ImageFolder("/home/comp5/ARTEK/SYZ_25/ILK_GOREV/VERI/altair/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = models.resnet34(pretrained=True)
# model = models.resnet18(pretrained=True)
# model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

best_val_loss = np.inf
early_stop_counter = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    correct = 0

    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]", leave=False)
    for images, labels in train_bar:
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()

        train_bar.set_postfix(loss=loss.item())

    train_acc = correct / len(train_dataset)
    train_loss /= len(train_dataset)

    model.eval()
    val_loss = 0
    correct = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]", leave=False)
    with torch.no_grad():
        for images, labels in val_bar:
            images = images.to(device)
            labels = labels.to(device).float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            preds = (torch.sigmoid(outputs) > 0.5).float()
            correct += (preds == labels).sum().item()

            val_bar.set_postfix(loss=loss.item())

    val_acc = correct / len(val_dataset)
    val_loss /= len(val_dataset)

    print(f"\n📊 Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_PATH)
        early_stop_counter = 0
        print("✓ En iyi model kaydedildi.")
    else:
        early_stop_counter += 1
        if early_stop_counter >= PATIENCE:
            print(f"🛑 Early stopping at epoch {epoch+1}")
            break

print("🎯 Eğitim tamamlandı. En iyi model kaydedildi:", MODEL_PATH)


In [ ]:
import os
import json
import math
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from torch import nn
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay
)

# ==== Ayarlar ====
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "/home/comp5/ARTEK/SYZ_25/SYZ_25_Egitim/ILK_GOREV/nihai_modeller/best_resnet50_yeniveri_teknofest_model/best_resnet50_yeniveri_model.pth"
TEST_DIR = "/home/comp5/ARTEK/SYZ_25/SYZ_25_Egitim/genel_veri/2025_Veri/test_veri_seti/ÜNİVERSİTEVEÜZERİ_EGİTİM 2_PNG/BT_TestSet/teknofest_test_hazir/test"
OUT_DIR = Path("denenm_dogrulama")
BATCH_SIZE = 32
THRESHOLD = 0.5
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ==== Transform ====
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ==== Dataset & Loader ====
test_dataset = datasets.ImageFolder(TEST_DIR, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
class_names = test_dataset.classes
file_paths = [s[0] for s in test_dataset.samples]

# ==== Model ====
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 1)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# ==== Tahminler ====
all_probs, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE).float().unsqueeze(1)
        outputs = model(images)
        probs = torch.sigmoid(outputs)
        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_probs = np.array(all_probs).flatten()
all_labels = np.array(all_labels).flatten()
all_preds = (all_probs >= THRESHOLD).astype(float)

# ==== Threshold Optimizasyonu ====
def find_best_threshold(y_true, y_probs):
    best_thresh = 0.5
    best_score = 0
    for thresh in np.linspace(0.1, 0.9, 100):
        preds = (y_probs >= thresh).astype(int)
        score = f1_score(y_true, preds, average='macro')
        if score > best_score:
            best_score = score
            best_thresh = thresh
    return best_thresh, best_score

best_thresh, best_score = find_best_threshold(all_labels, all_probs)
all_preds_best = (all_probs >= best_thresh).astype(int)

# ==== Metrikler ====
def compute_metrics(y_true, y_pred, y_probs, threshold):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    roc_auc = roc_auc_score(y_true, y_probs)
    cm = confusion_matrix(y_true, y_pred)
    class0_acc = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    class1_acc = cm[1, 1] / (cm[1, 0] + cm[1, 1])

    return {
        "accuracy": float(acc),
        "precision": float(prec),
        "recall": float(rec),
        "f1_score": float(f1),
        "macro_f1_score": float(macro_f1),
        "roc_auc": float(roc_auc),
        "class_0_acc": float(class0_acc),
        "class_1_acc": float(class1_acc),
        "threshold": float(threshold),
        "num_samples": int(len(y_true)),
        "class_labels": class_names
    }, cm

metrics_default, cm_default = compute_metrics(all_labels, all_preds, all_probs, THRESHOLD)
metrics_best, cm_best = compute_metrics(all_labels, all_preds_best, all_probs, best_thresh)

# ==== JSON Kayıt ====
with open(OUT_DIR / "metrics.json", "w") as f: json.dump(metrics_default, f, indent=2)
with open(OUT_DIR / "metrics_best.json", "w") as f: json.dump(metrics_best, f, indent=2)

# ==== Rapor CSV ====
report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True, zero_division=0)
pd.DataFrame(report).T.to_csv(OUT_DIR / "class_report.csv")

report_best = classification_report(all_labels, all_preds_best, target_names=class_names, output_dict=True, zero_division=0)
pd.DataFrame(report_best).T.to_csv(OUT_DIR / "class_report_best.csv")

# ==== Tahmin CSV ====
df = pd.DataFrame({
    "filepath": file_paths,
    "y_true": [class_names[int(x)] for x in all_labels],
    "y_pred_default": [class_names[int(x)] for x in all_preds],
    "y_pred_best": [class_names[int(x)] for x in all_preds_best],
    "prob_positive": all_probs
})
df.to_csv(OUT_DIR / "predictions.csv", index=False)

# ==== Confusion Matrix ====
for name, cm, t in [("default", cm_default, THRESHOLD), ("best", cm_best, best_thresh)]:
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix ({name.upper()} Threshold={t:.4f})")
    plt.savefig(OUT_DIR / f"confusion_matrix_{name}.png", dpi=150, bbox_inches="tight")
    plt.close()

# ==== ROC Curve ====
fpr, tpr, _ = roc_curve(all_labels, all_probs)
plt.figure()
plt.plot(fpr, tpr, label=f"AUC = {metrics_best['roc_auc']:.3f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.savefig(OUT_DIR / "roc_curve.png", dpi=150, bbox_inches="tight")
plt.close()

# ==== Görsel Grid ====
print("📷 Grid görseller kaydediliyor...")
def save_image_grid(preds, name):
    num_samples = len(file_paths)
    grid_size = 100
    num_chunks = (num_samples + grid_size - 1) // grid_size
    img_size = (224, 224)

    for chunk_idx in range(num_chunks):
        start = chunk_idx * grid_size
        end = min(start + grid_size, num_samples)
        chunk_paths = file_paths[start:end]
        chunk_true = all_labels[start:end]
        chunk_pred = preds[start:end]

        cols = int(round(math.sqrt(len(chunk_paths))))
        rows = math.ceil(len(chunk_paths) / cols)
        fig, axes = plt.subplots(rows, cols, figsize=(cols * 1.6, rows * 1.6))

        if rows == 1:
            axes = [axes]
        axes = np.array(axes).reshape(-1)

        for i, ax in enumerate(axes):
            idx = start + i
            if i < len(chunk_paths):
                img_path = chunk_paths[i]
                true_label = class_names[int(chunk_true[i])]
                pred_label = class_names[int(chunk_pred[i])]
                color = "green" if true_label == pred_label else "red"

                try:
                    img = Image.open(img_path).convert("RGB").resize(img_size)
                    ax.imshow(img)
                except:
                    ax.text(0.5, 0.5, "IMAGE ERROR", ha="center", va="center", fontsize=8)
                    ax.set_facecolor("gray")

                ax.set_title(f"T:{true_label}\nP:{pred_label}", fontsize=7, color=color)
            ax.axis("off")

        plt.tight_layout()
        out_file = OUT_DIR / f"samples_{name}_{chunk_idx+1}.png"
        plt.savefig(out_file, dpi=150, bbox_inches="tight")
        plt.close()
        print(f"✓ {out_file.name} kaydedildi.")

# Kaydet
save_image_grid(all_preds, "default")
save_image_grid(all_preds_best, "best")

print(f"🏁 Tamamlandı. Best threshold: {best_thresh:.4f}")
